<a href="https://colab.research.google.com/github/XingweiLiang/XingweiLiang.github.io/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

### Install PyTorch and torchaudio

In [1]:
import torch
print(torch.__version__)

2.0.1+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [2]:
! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [3]:
! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 MB 9.6 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [4]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.8
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [5]:
# ! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-ayzs7262
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-ayzs7262
  Resolved https://github.com/lhotse-speech/lhotse to commit 287de1dbadcb70bf7398d3d482c1f730d2c5407f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 12.9 MB/s eta 0:00:00
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.287de1d.clean-py3-none-any.whl size=729005 sha256=ad242482c2cbf0a22c3283b62898ef448ec7407eb4f7690af2ba748080d52ced
  Stored in directory: /tmp/pip-ephem-wheel-cache-lh1e2ab6/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
  Created wheel for intervaltree: filename=intervaltree-3

### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [6]:
! pwd

/content


In [7]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 14310, done.
remote: Counting objects: 100% (1427/1427), done.
remote: Compressing objects: 100% (524/524), done.
remote: Total 14310 (delta 892), reused 1237 (delta 801), pack-reused 12883
Receiving objects: 100% (14310/14310), 16.58 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (9720/9720), done.


Now install dependencies of `icefall`:

In [9]:
! cd icefall && \
  pip install -r requirements.txt

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [10]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0


In [11]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

2023-10-06 05:10:49 (prepare.sh:27:main) dl_dir: /content/icefall/egs/yesno/ASR/download
2023-10-06 05:10:49 (prepare.sh:30:main) Stage 0: Download data
/content/icefall/egs/yesno/ASR/download/waves_yesno.tar.gz: 100% 4.70M/4.70M [00:01<00:00, 4.01MB/s]
2023-10-06 05:10:54 (prepare.sh:39:main) Stage 1: Prepare yesno manifest
2023-10-06 05:10:57 (prepare.sh:45:main) Stage 2: Compute fbank for yesno
2023-10-06 05:11:00,052 INFO [compute_fbank_yesno.py:65] Processing train
Extracting and storing features: 100% 90/90 [00:00<00:00, 168.90it/s]
2023-10-06 05:11:00,610 INFO [compute_fbank_yesno.py:65] Processing test
Extracting and storing features: 100% 30/30 [00:00<00:00, 293.60it/s]
2023-10-06 05:11:01 (prepare.sh:51:main) Stage 3: Prepare lang
2023-10-06 05:11:07,249 INFO [prepare_lang_fst.py:174] Building standard CTC topology
2023-10-06 05:11:07,250 INFO [prepare_lang_fst.py:183] Building L
2023-10-06 05:11:07,250 INFO [prepare_lang_fst.py:191] Building HL
2023-10-06 05:11:07,251 INFO [

## Training

In [12]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/train.py

2023-10-06 05:12:19,770 INFO [train.py:481] Training started
2023-10-06 05:12:19,770 INFO [train.py:482] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'lr': 0.01, 'feature_dim': 23, 'weight_decay': 1e-06, 'start_epoch': 0, 'best_train_loss': inf, 'best_valid_loss': inf, 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 10, 'reset_interval': 20, 'valid_interval': 10, 'beam_size': 10, 'reduction': 'sum', 'use_double_scores': True, 'world_size': 1, 'master_port': 12354, 'tensorboard': True, 'num_epochs': 15, 'seed': 42, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'e400fa3b456faf8afe0ee5bfe572946b4921a3db', 'k2-git-

## Decoding

In [13]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

2023-10-06 05:13:25,509 INFO [decode.py:262] Decoding started
2023-10-06 05:13:25,509 INFO [decode.py:263] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'feature_dim': 23, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 14, 'avg': 2, 'export': False, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.3', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'e400fa3b456faf8afe0ee5bfe572946b4921a3db', 'k2-git-date': 'Sat Jul 15 04:21:50 2023', 'lhotse-version': '1.17.0.dev+git.287de1d.clean', 'torch-version': '2.0.1+cu118', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'maste

### Show the decoding result

In [14]:
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

0_0_0_1_0_0_0_1-0:	ref=['NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_0_1_0_0_0_1-0:	hyp=['NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_0_0_0_1_0-1:	ref=['NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO']
0_0_1_0_0_0_1_0-1:	hyp=['NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO']
0_0_1_0_0_1_1_1-2:	ref=['NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']
0_0_1_0_0_1_1_1-2:	hyp=['NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']
0_0_1_0_1_0_0_1-3:	ref=['NO', 'NO', 'YES', 'NO', 'YES', 'NO', 'NO', 'YES']
0_0_1_0_1_0_0_1-3:	hyp=['NO', 'NO', 'YES', 'NO', 'YES', 'NO', 'NO', 'YES']
0_0_1_1_0_0_0_1-4:	ref=['NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_1_0_0_0_1-4:	hyp=['NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES']
0_0_1_1_0_1_1_0-5:	ref=['NO', 'NO', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO']
0_0_1_1_0_1_1_0-5:	hyp=['NO', 'NO', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO']
0_0_1_1_1_0_0_0-6:	ref=['NO', 'NO', 'YES', 'YES', 'YES', 'NO', 'NO', 'NO']
0_0_1_1_1_0_0_0-6:	hyp=['

### Show the detailed WER

In [15]:
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

%WER = 0.42
Errors: 0 insertions, 1 deletions, 0 substitutions, over 240 reference words (239 correct)
Search below for sections starting with PER-UTT DETAILS:, SUBSTITUTIONS:, DELETIONS:, INSERTIONS:, PER-WORD STATS:

PER-UTT DETAILS: corr or (ref->hyp)  
0_0_0_1_0_0_0_1-0:	NO NO NO YES NO NO NO YES
0_0_1_0_0_0_1_0-1:	NO NO YES NO NO NO YES NO
0_0_1_0_0_1_1_1-2:	NO NO YES NO NO YES YES YES
0_0_1_0_1_0_0_1-3:	NO NO YES NO YES NO NO YES
0_0_1_1_0_0_0_1-4:	NO NO YES YES NO NO NO YES
0_0_1_1_0_1_1_0-5:	NO NO YES YES NO YES YES NO
0_0_1_1_1_0_0_0-6:	NO NO YES YES YES NO NO NO
0_0_1_1_1_1_0_0-7:	NO NO YES YES YES YES NO NO
0_1_0_0_0_1_0_0-8:	NO YES NO NO NO YES NO NO
0_1_0_0_1_0_1_0-9:	NO YES NO NO YES NO YES NO
0_1_0_1_0_0_0_0-10:	NO YES NO YES NO NO NO (NO->*)
0_1_0_1_1_1_0_0-11:	NO YES NO YES YES YES NO NO
0_1_1_0_0_1_1_1-12:	NO YES YES NO NO YES YES YES
0_1_1_1_0_0_1_0-13:	NO YES YES YES NO NO YES NO
0_1_1_1_1_0_1_0-14:	NO YES YES YES YES NO YES NO
1_0_0_0_0_0_0_0-15:	YES NO NO NO NO NO

# Pre-trained model

### Download the pre-trained model

In [16]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir tmp && \
  cd tmp && \
  git lfs install && \
  git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn

Updated git hooks.
Git LFS initialized.
Cloning into 'icefall_asr_yesno_tdnn'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), 2.28 MiB | 7.37 MiB/s, done.


In [17]:
! sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [18]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir -p tmp && \
  tree tmp

/bin/bash: line 1: tree: command not found


In [19]:
! sudo apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (85.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 120879 files and directories currently instal

In [20]:
! cd /content/icefall/egs/yesno/ASR && \
  mkdir -p tmp && \
  tree tmp

tmp
└── icefall_asr_yesno_tdnn
    ├── lang_phone
    │   ├── HLG.pt
    │   ├── L_disambig.pt
    │   ├── lexicon_disambig.txt
    │   ├── lexicon.txt
    │   ├── Linv.pt
    │   ├── L.pt
    │   ├── tokens.txt
    │   └── words.txt
    ├── lm
    │   ├── G.arpa
    │   └── G.fst.txt
    ├── pretrained.pt
    ├── README.md
    └── test_waves
        ├── 0_0_0_1_0_0_0_1.wav
        ├── 0_0_1_0_0_0_1_0.wav
        ├── 0_0_1_0_0_1_1_1.wav
        ├── 0_0_1_0_1_0_0_1.wav
        ├── 0_0_1_1_0_0_0_1.wav
        ├── 0_0_1_1_0_1_1_0.wav
        ├── 0_0_1_1_1_0_0_0.wav
        ├── 0_0_1_1_1_1_0_0.wav
        ├── 0_1_0_0_0_1_0_0.wav
        ├── 0_1_0_0_1_0_1_0.wav
        ├── 0_1_0_1_0_0_0_0.wav
        ├── 0_1_0_1_1_1_0_0.wav
        ├── 0_1_1_0_0_1_1_1.wav
        ├── 0_1_1_1_0_0_1_0.wav
        ├── 0_1_1_1_1_0_1_0.wav
        ├── 1_0_0_0_0_0_0_0.wav
        ├── 1_0_0_0_0_0_1_1.wav
        ├── 1_0_0_1_0_1_1_1.wav
        ├── 1_0_1_1_0_1_1_1.wav
        ├── 1_0_1_1_1_1_0_1.wav
        ├── 1_1

In [21]:
! cd /content/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

/bin/bash: line 1: soxi: command not found


In [22]:
! sudo apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 18 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 amd64 14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1 [240 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 

In [23]:
! cd /content/icefall/egs/yesno/ASR && \
  soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav


Input File     : 'tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav'
Channels       : 1
Sample Rate    : 8000
Precision      : 16-bit
Duration       : 00:00:06.76 = 54080 samples ~ 507 CDDA sectors
File Size      : 108k
Bit Rate       : 128k
Sample Encoding: 16-bit Signed Integer PCM



## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [24]:
! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

Looking in links: https://csukuangfj.github.io/kaldifeat/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.0/574.0 kB 6.9 MB/s eta 0:00:00


## Inference with a pre-trained model


### View help information

In [25]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py --help

usage: pretrained.py
       [-h]
       --checkpoint
       CHECKPOINT
       --words-file
       WORDS_FILE
       --HLG
       HLG
       sound_files
       [sound_files ...]

positional arguments:
  sound_files
    The input
    sound
    file(s) to
    transcribe.
    Supported
    formats are
    those
    supported
    by torchaud
    io.load().
    For
    example,
    wav and
    flac are
    supported.

options:
  -h, --help
    show this
    help
    message and
    exit
  --checkpoint CHECKPOINT
    Path to the
    checkpoint.
    The
    checkpoint
    is assumed
    to be saved
    by icefall.
    checkpoint.
    save_checkp
    oint(). You
    can use ./t
    dnn/export.
    py to
    obtain it.
    (default:
    None)
  --words-file WORDS_FILE
    Path to
    words.txt
    (default:
    None)
  --HLG HLG
    Path to
    HLG.pt.
    (default:
    None)


### Decode a single sound file

In [26]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

2023-10-06 05:27:08,084 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp/icefall_asr_yesno_tdnn/pretrained.pt', 'words_file': './tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt', 'HLG': './tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt', 'sound_files': ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav']}
2023-10-06 05:27:08,103 INFO [pretrained.py:142] device: cuda:0
2023-10-06 05:27:08,103 INFO [pretrained.py:144] Creating model
2023-10-06 05:27:09,749 INFO [pretrained.py:156] Loading HLG from ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt
2023-10-06 05:27:09,751 INFO [pretrained.py:160] Constructing Fbank computer
2023-10-06 05:27:09,752 INFO [pretrained.py:170] Reading sound files: ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav']
2023-10-06 05:27:09,753 INFO [pretrained.py:176] Decoding s

### Decode multiple sound files

In [27]:
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
    --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
    --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
    ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav

2023-10-06 05:27:52,355 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp/icefall_asr_yesno_tdnn/pretrained.pt', 'words_file': './tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt', 'HLG': './tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt', 'sound_files': ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav', './tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav']}
2023-10-06 05:27:52,374 INFO [pretrained.py:142] device: cuda:0
2023-10-06 05:27:52,374 INFO [pretrained.py:144] Creating model
2023-10-06 05:27:54,246 INFO [pretrained.py:156] Loading HLG from ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt
2023-10-06 05:27:54,249 INFO [pretrained.py:160] Constructing Fbank computer
2023-10-06 05:27:54,249 INFO [pretrained.py:170] Reading sound files: ['./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wa